In [13]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import hackathon.prompts as prompts
import hackathon.functions as f
from dotenv import load_dotenv
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
import os
import math


# For the agents
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import langchain.llms as llms
import time
import json
from flask import Flask, request, jsonify

#For the main OPENAI ASSISTANT
import openai as openai
#
load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [14]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
trx_path = os.environ["trx_df_path"]

In [15]:
def get_UN_data():
    df = pd.read_csv(trx_path)
    df["date"] = pd.to_datetime(df["date"])
    return df.set_index("user")


df = get_UN_data().loc["user_3"].reset_index(drop=True)
COLUMNS = df.columns

In [16]:
f.create_temporal_features(df, "date")

,date,amount,trx_category,method,team,receipt,VAT,Status,day_of_week,day_of_year,week_of_year,week_of_month,is_weekday,month,year
0,2020-11-19,7,Restaurants,taxes,Marketing,Not needed,Missing,Executed,3,324,47,3,True,11,2020
1,2020-11-10,57,Mobile,direct_debit,Finance,Not needed,Missing,Executed,1,315,46,2,True,11,2020
2,2021-07-30,302,Sport,direct_debit,IT,Missing,Filled in,Processing,4,211,30,5,True,7,2021
3,2020-07-15,431,Hotel,taxes,Sales,Missing,Missing,Declined,2,197,29,3,True,7,2020
4,2021-03-30,106,Fees,transfer,Sales,Missing,Missing,Declined,1,89,13,5,True,3,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,2020-07-26,538,Hotel,transfer,Sales,Attached,Missing,Declined,6,208,30,4,False,7,2020
323,2020-09-03,207,Health,direct_debit,Sales,Missing,Missing,Processing,3,247,36,1,True,9,2020
324,2020-03-22,902,Fees,transfer,Finance,Attached,Filled in,Processing,6,82,12,4,False,3,2020
325,2020-12-30,401,Train,card,Finance,Missing,Missing,Executed,2,365,53,5,True,12,2020


In [17]:
# Create the agent
client = openai.OpenAI(api_key=OPENAI_API_KEY)
assistant_id = f.create_assistant(client)
thread = f.start_conversation(client)

Loaded existing assistant ID.
Starting a new conversation...
New thread created with ID: thread_w687B3iaXTZdmxjnzPBkkhrr


In [24]:
thread = f.start_conversation(client)

prompt = "Show me marketing weekend transactions"
# prompt = "show 2021 marketing transactions without a receipt"
res = f.chat(client, thread["thread_id"], prompt, df[COLUMNS], assistant_id)

Starting a new conversation...
New thread created with ID: thread_StYFbiKcKtf9rIrQaaZCInc9
Received message: Show me marketing weekend transactions for thread ID: thread_StYFbiKcKtf9rIrQaaZCInc9
What is the average amount, number of transactions and most frequent trx_category for the Marketing team during weekends?


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df_marketing_weekend = df[(df['team'] == 'Marketing') & (df['is_weekday'] == False)]\navg_amount = df_marketing_weekend['amount'].mean()\nnum_transactions = df_marketing_weekend.shape[0]\nmost_frequent_category = df_marketing_weekend['trx_category'].mode()[0]\navg_amount, num_transactions, most_frequent_category"}`


(592.8125, 16, 'Taxi')The average amount for transactions made by the Marketing team during weekends is 592.81. The number of transactions made by the Marketing team during weekends is 16. The most frequent transaction category for the Marketing team during weekends is "Taxi".


In [21]:
res["response"]

'The Marketing team made 16 transactions during weekends with an average transaction amount of $592.81. The most frequent transaction category was "Taxi". Here are the filtered weekend transactions for the Marketing team.'

In [23]:
res["data_filterered"].describe()

,amount,day_of_week,day_of_year,week_of_year,week_of_month,month,year
count,16.0,16.0,16.0,16.0,16.0,16.0,16.0
mean,592.8125,5.6875,192.25,30.75,2.6875,6.8125,2020.5
std,244.020004,0.478714,111.588829,15.381807,1.493039,3.581783,0.516398
min,151.0,5.0,3.0,3.0,1.0,1.0,2020.0
25%,447.5,5.0,116.25,21.0,1.0,4.75,2020.0
50%,622.5,6.0,199.5,32.5,2.5,7.0,2020.5
75%,771.0,6.0,264.5,42.5,4.0,9.25,2021.0
max,921.0,6.0,359.0,53.0,5.0,12.0,2021.0


In [ ]:
f.create_temporal_features(df[COLUMNS].reset_index(drop=True), "date").query(
    "year == 2021"
).team.value_counts()

Sales        37
HR           34
IT           31
Marketing    31
Finance      25
Name: team, dtype: int64